# <a id='toc1_'></a>[Statistics & Public Health 2: Data Analysis](#toc0_)

Sam Celarek  
Data Science   
scelarek@gmail.com  

June 4th, 2023


# <a id='toc2_'></a>[1. Introduction](#toc0_)

In this project, we will perform a set of analyses on the relationship between different variables and the mosquito number, as well as the probability of finding West Nile Virus (WNV) at any particular time and location. 


## <a id='toc2_1_'></a>[1.1. Key Questions](#toc0_)


# <a id='toc3_'></a>[2. Setup and Data Collection](#toc0_)

We will be utilizing the cleaned mosquito tracking data from the city of Chicago, Illinois, spanning from 2008 to 2019 provided [here](link_to_dataset). This section will include the necessary libraries and modules for the analysis, as well as the data preparation steps.


In [53]:
import covsirphy as cs
cs.__version__


'3.0.0'

In [54]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import re
from functools import reduce

from collections import defaultdict
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import functools
from IPython.display import display, Markdown
import math
import os
from pprint import pprint
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.ticker import ScalarFormatter
%matplotlib inline
import dask.dataframe as dd
pd.plotting.register_matplotlib_converters()
import scipy as sci
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import sympy as sym

# initialize styling params
# Ramdam

np.random.seed(123)
os.environ["PYTHONHASHSEED"] = "123"
# Matplotlib
plt.style.use("seaborn-ticks")
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["ytick.direction"] = "in"
plt.rcParams["font.size"] = 11.0
plt.rcParams["figure.figsize"] = (9, 6)
# Pandas
pd.set_option("display.max_colwidth", 1000)

# Load the data

# Display the first few rows of the dataframe


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Samsickle\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Samsickle\AppData\Local\Temp\ipykernel_25064\2301690978.py", line 23, in <module>
    pd.plotting.register_matplotlib_converters()
  File "C:\Users\Samsickle\AppData\Roaming\Python\Python39\site-packages\pandas\plotting\_misc.py", line 64, in register
    plot_backend = _get_plot_backend("matplotlib")
  File "C:\Users\Samsickle\AppData\Roaming\Python\Python39\site-packages\pandas\plotting\_core.py", line 1907, in _get_plot_backend
  File "C:\Users\Samsickle\AppData\Roaming\Python\Python39\site-packages\pandas\plotting\_core.py", line 1838, in _load_backend
    def _get_plot_backend(backend: str | None = None):
ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

During handling of the above exception, anot

In [55]:
# folder_holder = "C://Users/Samsickle/Documents/BrainStation_Capstone/Data/"

# # # time series data
# epid_df = pd.read_csv(f'{folder_holder}epidemiology.csv') # 1
# hospitalizations_df = pd.read_csv(f'{folder_holder}hospitalizations.csv') # 2
# vac_df = pd.read_csv(f'{folder_holder}vaccinations.csv') # 3
# mobility_df = pd.read_csv(f'{folder_holder}mobility.csv') # 4
# gov_response_df = pd.read_csv(f'{folder_holder}oxford-government-response.csv') # 5
# weather_df = pd.read_csv(f'{folder_holder}weather.csv') # 6

# timeland_df = [epid_df, hospitalizations_df, mobility_df, vac_df, gov_response_df, weather_df]

SIR_df = pd.read_csv(f'{folder_holder}US Data.csv') # 2

In [56]:
SIR_df.head()

,id,date,confirmed,deaths,recovered,tests,vaccines,people_vaccinated,people_fully_vaccinated,hosp,...,iso_alpha_3,iso_alpha_2,iso_numeric,iso_currency,key_local,key_google_mobility,key_apple_mobility,key_jhu_csse,key_nuts,key_gadm
0,59a13ceb,2020-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,USA,US,840,USD,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,United States,US,NaN,USA
1,59a13ceb,2020-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,USA,US,840,USD,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,United States,US,NaN,USA
2,59a13ceb,2020-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,USA,US,840,USD,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,United States,US,NaN,USA
3,59a13ceb,2020-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,USA,US,840,USD,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,United States,US,NaN,USA
4,59a13ceb,2020-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,USA,US,840,USD,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,United States,US,NaN,USA


In [57]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df):
    # Filter rows based on column: 'location_key'
    # df = df[df['location_key'].str.contains(r"^(US|CA|DE|GB|FR|JP|AU|BR|ZA|IN)$", na=False)]
    df = df[df['location_key'].str.contains(r"^(US)$", na=False)] # only the us to start

    # Change column type to datetime64[ns] for column: 'date'
    try:
        df = df.astype({'date': 'datetime64[ns]'})
    except:
        pass
    # Change column type to category for column: 'location_key'
    df = df.astype({'location_key': 'category'})
    return df


# time_series_dfs = list(map(clean_data, timeland_df))

# # Assume dfs is your list of dataframes
# time_series_dfs = reduce(lambda left,right: pd.merge(left,right,on=['location_key', 'date'], how='left'), time_series_dfs).copy()


In [58]:
# time_series_dfs.to_pickle('../Data/time_series_dfs.pkl')

time_series_dfs = pd.read_pickle('../Data/time_series_dfs.pkl')

time_series_dfs.sample(3)


,date,location_key,new_confirmed,new_deceased,new_recovered,new_tested,cumulative_confirmed,cumulative_deceased,cumulative_recovered,cumulative_tested,...,facial_coverings,vaccination_policy,stringency_index,average_temperature_celsius,minimum_temperature_celsius,maximum_temperature_celsius,rainfall_mm,snowfall_mm,dew_point,relative_humidity
421,2021-02-25,US,67307.0,2860.0,NaN,1484532.0,27652192.0,491128.0,NaN,345098603.0,...,4.0,1.0,68.06,0.777778,-3.850000,6.855556,0.0,88.9,-4.533333,68.183734
904,2022-06-23,US,104031.0,497.0,NaN,NaN,84077044.0,971234.0,NaN,NaN,...,2.0,5.0,38.89,24.694444,19.638889,32.616667,0.0,NaN,19.266667,72.097510
275,2020-10-02,US,50488.0,902.0,NaN,976617.0,7181841.0,190501.0,NaN,122496961.0,...,4.0,0.0,62.50,10.705556,0.816667,22.955556,0.0,NaN,-0.572222,45.854731


In [59]:
folder_holder = "C://Users/Samsickle/Documents/BrainStation_Capstone/Data/"
# C:\Users\Samsickle\Documents\BrainStation_Capstone\Data

# # # time series data
epid_df = pd.read_csv(f'{folder_holder}epidemiology.csv') # 1
# hospitalizations_df = pd.read_csv(f'{folder_holder}hospitalizations.csv') # 2
# vac_df = pd.read_csv(f'{folder_holder}vaccinations.csv') # 3
# mobility_df = pd.read_csv(f'{folder_holder}mobility.csv') # 4
# gov_response_df = pd.read_csv(f'{folder_holder}oxford-government-response.csv') # 5
# weather_df = pd.read_csv(f'{folder_holder}weather.csv') # 6

In [60]:
epid_df.sample(5)

,date,location_key,new_confirmed,new_deceased,new_recovered,new_tested,cumulative_confirmed,cumulative_deceased,cumulative_recovered,cumulative_tested
9264652,2021-01-03,PE_LMA_0117,40.0,3.0,NaN,NaN,14258.0,1402.0,NaN,NaN
8378138,2022-01-24,IT_42_SP,252.0,NaN,NaN,NaN,35969.0,NaN,NaN,NaN
8221171,2021-09-29,IN_TN_TKS,2.0,0.0,6.0,0.0,27277.0,484.0,26734.0,26338.0
9914687,2021-11-16,US_AK_02275,2.0,0.0,NaN,NaN,188.0,0.0,NaN,NaN
6369207,2021-03-20,DE_BW_08222,91.0,0.0,91.0,NaN,11727.0,266.0,11461.0,NaN


Data Wireframe:

1. Date and Location:
    - 'date' - the day of the observations
    - 'location_key' - the country of the observations. I choose ten countries of interest in different regions

United States (US)  
Canada (CA)  
Germany (DE)  
United Kingdom (GB)  
France (FR)  
Japan (JP)  
Australia (AU)  
Brazil (BR)  
South Africa (ZA)  
India (IN)  
These countries are often used as indicators for their respective regions due to their significant economic influence, political stability, and comprehensive data collection practices.

2. COVID-19 Statistics:
    - 'new_confirmed' (New Positive Cases) - the number of new confirmed cases of COVID-19, this includes some negative numbers to account for data corrections in the previous days, however because these numbers are aggregated accross a whole country, the negative numbers are often very small and remain positive
    - 'new_deceased' (New Deaths) - the number of new deaths due to COVID-19 also shares the negative number problem
    - 'new_hospitalized_patients' (New Hospitalizations) 

3. Mobility Data:
    - 'mobility_retail_and_recreation'
    - 'mobility_grocery_and_pharmacy'
    - 'mobility_parks'
    - 'mobility_transit_stations'
    - 'mobility_workplaces'
    - 'mobility_residential'



4. Vaccination Data:
    - 'new_persons_vaccinated'
    - 'cumulative_persons_vaccinated'
    - 'new_persons_fully_vaccinated'
    - 'cumulative_persons_fully_vaccinated'
    - 'new_vaccine_doses_administered'
    - 'cumulative_vaccine_doses_administered'

5. Policy Measures:
    - 'school_closing'
    - 'workplace_closing'
    - 'cancel_public_events'
    - 'restrictions_on_gatherings'
    - 'public_transport_closing'
    - 'stay_at_home_requirements'
    - 'restrictions_on_internal_movement'
    - 'international_travel_controls'
    - 'income_support'
    - 'debt_relief'
    - 'fiscal_measures'
    - 'international_support'
    - 'public_information_campaigns'
    - 'testing_policy'
    - 'contact_tracing'
    - 'emergency_investment_in_healthcare'
    - 'investment_in_vaccines'
    - 'facial_coverings'
    - 'vaccination_policy'
    - 'stringency_index'

6. Weather Data:
    - 'average_temperature_celsius'
    - 'minimum_temperature_celsius'
    - 'maximum_temperature_celsius'
    - 'rainfall_mm'
    - 'snowfall_mm'
    - 'dew_point'
    - 'relative_humidity'


In [61]:
# location dfs
geography_df = pd.read_csv(f'{folder_holder}geography.csv') #1
health_df = pd.read_csv(f'{folder_holder}health.csv') #2
demographics_df = pd.read_csv(f'{folder_holder}demographics.csv') #3
economics_df = pd.read_csv(f'{folder_holder}economy.csv') #4


locationland_df = [geography_df, health_df, demographics_df, economics_df]

In [62]:

location_df = list(map(clean_data, locationland_df))

# Assume dfs is your list of dataframes
location_df = reduce(lambda left,right: pd.merge(left,right,on='location_key', how='left'), location_df).copy()

location_df


C:\Users\Samsickle\AppData\Local\Temp\ipykernel_25064\3449870818.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[df['location_key'].str.contains(r"^(US)$", na=False)] # only the us to start
C:\Users\Samsickle\AppData\Local\Temp\ipykernel_25064\3449870818.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[df['location_key'].str.contains(r"^(US)$", na=False)] # only the us to start
C:\Users\Samsickle\AppData\Local\Temp\ipykernel_25064\3449870818.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[df['location_key'].str.contains(r"^(US)$", na=False)] # only the us to start
C:\Users\Samsickle\AppData\Local\Temp\ipykernel_25064\3449870818.py:7: UserWarning: This pattern is interpreted as a regular ex

,location_key,openstreetmap_id,latitude,longitude,elevation_m,area_sq_km,area_rural_sq_km,area_urban_sq_km,life_expectancy,smoking_prevalence,...,population_age_20_29,population_age_30_39,population_age_40_49,population_age_50_59,population_age_60_69,population_age_70_79,population_age_80_and_older,gdp_usd,gdp_per_capita_usd,human_capital_index
0,US,148838.0,39.828175,-98.5795,NaN,9831510.0,8549545.0,802053.0,78.539024,21.8,...,46094075.0,44668270.0,40348397.0,42120077.0,38488170.0,24082597.0,13147182.0,2.137442e+13,65118.0,0.762


In [63]:

location_df.to_pickle('../Data/location_df.pkl')
location_df = pd.read_pickle('../Data/location_df.pkl')

location_df.sample(3)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Samsickle\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Samsickle\AppData\Local\Temp\ipykernel_25064\2214498508.py", line 4, in <module>
    location_df.sample(3)
  File "C:\Users\Samsickle\AppData\Roaming\Python\Python39\site-packages\pandas\core\generic.py", line 5773, in sample
    A new object of same type as caller containing `n` items randomly
  File "C:\Users\Samsickle\AppData\Roaming\Python\Python39\site-packages\pandas\core\sample.py", line 150, in sample
  File "mtrand.pyx", line 965, in numpy.random.mtrand.RandomState.choice
ValueError: Cannot take a larger sample than population when 'replace=False'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Samsickle\AppData\Roaming\Python\Python39\site-packages\IPython\core\interacti

This dataframe provides a comprehensive snapshot of COVID-19 data, mobility metrics, government restrictions, and weather conditions for specific locations on specific dates. Here's a brief overview of the columns:

1. `Entry ID`: A unique identifier for each row in the dataframe.
2. `Date`: The date for the day on which the data was recorded.
3. `Location Key`: A code representing the location (10 different countries in total) for which the data is reported.

4. `New Confirmed`: The number of new confirmed COVID-19 cases on the given date.
5. `New Deceased`: The number of new COVID-19 related deaths on the given date.
6. `New Recovered`: The number of new recoveries from COVID-19 on the given date.
7. `New Tested`: The number of new COVID-19 tests conducted on the given date.

8. `New Hospitalizations`: The number of new hospitalizations due to COVID-19 on the given date.
9. `Current Hospitalizations`: The total number of current hospitalizations due to COVID-19 on the given date.

10. `New Fully Vaccinated (29+ other Vaccination Columns)`: The number of new fully vaccinated individuals on the given date. There are 29 other columns related to vaccination data here too.

11. `Retail and Recreation Mobility (5+ other Mobility Metrics)`: A measure of mobility in retail and recreation spaces, along with 5 other columns related to different aspects of mobility.

12. `School Closing (19+ other Government Restrictions)`: A measure indicating whether schools were closed on the given date, along with 19 other columns related to different government restrictions.

13. `Average Temp (6+ Other Weather Columns)`: The average temperature on the given date, along with 6 other columns related to different weather conditions.

In total there are 9880 and 82 rows for 6.3mbs of data. The main way I could increase or decrease the size of the dataset would be to include more countries, regions, or counties in the analysis. For now this is my starter df.
